<a href="https://colab.research.google.com/github/WuilsonEstacio/python-for-data-science/blob/main/Medicion_de_credito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mediciones de crédito

En este proyecto se propone la construcción de modelo predictivo que permita determinar el comportamiento de un buen o mal cliente, a partir de su historial de pago de un crédito.

Para lograr esto, se parte de una base de datos anónima de una entidad bancaria. Con esta base de datos se sigue el siguiente flujo de trabajo (El flujo de trabajo se encuentra desarrolado en el archivo credit_score.ipynb ):

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('Anonimizada_DataBase.txt', delimiter = '|', decimal=",")
df.head()

In [ ]:
df.info()


Se deben hacer cambios de tipo de datos en algunas columnas. La columna Periodo, debe tener formato de fecha y las columnas Cupo y Saldo y DIAS0 deben ser tipo float.

In [ ]:
#Resultados en meses 
# df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m', errors='coerce').dt.to_period('m')
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m', errors='coerce')

In [ ]:
# convertiremos las columnas tipo objet a float
columns = ['Cupo', 'Saldo_Mora1', 'Saldo_Mora2', 'Saldo_Mora3', 'Saldo_Mora4', 
           'Saldo_Mora5', 'Saldo_Mora6', 'Saldo1', 'Saldo2', 'Saldo3', 'Saldo4', 
           'Saldo5', 'Saldo6']
for colum in columns:
    df[colum] = pd.to_numeric(df[colum], downcast='float', errors = 'coerce').astype(float)

In [ ]:
# convertiremos la columna dia0 en flotante
df['DIAS0'] = df['DIAS0'].astype(float)

In [ ]:
# para ver que tipo de variables tiene las columnas
categori_columna_x=df.GENERO.unique()
categori_columna_x

In [ ]:
df.info()

Index. Se define el index de la base de datos con la columna Identificador. Las entradas de campo deben ser únicas, así que se deben hacer un filtrado

In [ ]:
# se observa la cantidad de datos de entrada en el id
print('Se tienen', df['Identificador'].count(), 'entradas.')

In [ ]:
#  se mira cuales no estan repetidas
print('Solo', df['Identificador'].nunique(), 'don únicas')

In [ ]:
# Se Eliminan datos repetidos
df = df.drop_duplicates(subset=['Identificador'],  keep= False)

In [ ]:
# se confirma la cantidad de datos de entrada
print('Se tienen', df['Identificador'].count(), 'entradas.')

In [ ]:
# se confirma la cantidad de datos no repetidos como son iguales no hay datos repetidos
print('Solo', df['Identificador'].nunique(), 'don únicas')

In [ ]:
#Se elije la comuna Identificador como Index 
df = df.set_index('Identificador')

In [ ]:
#Se eliminan entradas con valores Nan
df = df.dropna()

In [ ]:
df.head()

In [ ]:
# para ver el Número de datos ausentes por variable
df.isnull().sum().sort_values(ascending=False,)[:21]

In [ ]:
#Información de Data Set
df.info()

In [ ]:
# observamos longitud de la base de datos
print('La base de datos tiene:', len(df), 'entradas.')


# 2. Descriptive Statistics

In [ ]:
df.describe()

In [ ]:
df.describe().T

In [ ]:
# df_features = pd.DataFrame([])
# Miramos Saldo promedio de los ultimos 6 meses, con la media aritmetica de cada fila
df_features['Saldo_Promedio'] = df[['Saldo1','Saldo2','Saldo3','Saldo4','Saldo5','Saldo6']].mean(axis=1)

In [ ]:
df_features['Flag_Genero'] = df.GENERO.eq('F').mul(1)

# Porcentaje promedio de saldo en mora con respecto al cupo en los 6 meses.

In [ ]:
s_m_prom = df[['Saldo_Mora1','Saldo_Mora2','Saldo_Mora3','Saldo_Mora4','Saldo_Mora5','Saldo_Mora6']].mean(axis=1)
df_features['Porc_prom_Saldo_mora'] = s_m_prom/ df['Cupo']
#Cuando s_m_max es 0 significa que no tiene saldo de mora, por lo tanto el porcentaje promedio de saldo 
#en mora es 0
df_features['Porc_prom_Saldo_mora'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_features['Porc_prom_Saldo_mora'] = df_features['Porc_prom_Saldo_mora'].fillna(0)*100

In [ ]:
s_m_max = df[['Saldo_Mora1','Saldo_Mora2','Saldo_Mora3','Saldo_Mora4','Saldo_Mora5','Saldo_Mora6']].max(axis=1)
df_features['Porc_max_Saldo_mora'] = s_m_prom/ df['Cupo']
#Cuando s_m_max es 0 significa que no tiene saldo de mora, por lo tanto el porcentaje promedio de saldo 
#en mora es 0
df_features['Porc_max_Saldo_mora'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_features['Porc_max_Saldo_mora'] = df_features['Porc_max_Saldo_mora'].fillna(0)*100

In [ ]:
#Cuando s_m_max es 0 significa que no tiene saldo de mora, por lo tanto el porcentaje promedio de saldo 
#en mora es 0
df_features['Porc_prom_Saldo_mora'].replace([np.inf, -np.inf], np.nan, inplace=True)
df_features['Porc_prom_Saldo_mora'] = df_features['Porc_prom_Saldo_mora'].fillna(0)*100


# Utilización máxima de los últimos 6 meses
Se calcula la utilización de la cuenta como Saldo/Cupo.

In [ ]:
df['util_max_seis'] = df['Saldo6'] / df['Cupo'] # se hacen los calculo
df['util_max_seis'].replace([np.inf, -np.inf], np.nan, inplace=True) # se realizan los remplazos
df['util_max_seis'] = df['util_max_seis'].fillna(0) # se crea nueva columna

# Cupo promedio de los primeros 3 meses
Se calcula como Cupo/Número_Productos.

In [ ]:
df_features['Cupo_prom'] = df['Cupo']/df['Numero_Producto']

Antigüedad de la obligación en meses a 2019-08

In [ ]:
df['date'] = 201908
df['date'] = pd.to_datetime(df['date'], format='%Y%m', errors='coerce')

In [ ]:
# https://numpy.org/doc/stable/reference/arrays.datetime.html
df_features['Antiguedad'] = ((df.date - df.periodo)/np.timedelta64(1, 'M'))

# Segmentation Class
Se crean dos etiquetas para los clientes tomando 0 como clientes buenos y 1 como clientes malos

In [ ]:
#Clientes que que luego de los 6 meses hayan alcanzado un máximo de 30 días en mora
df_features['Tipo_Cliente_1'] = 0 #creamos casillas basias

DataFrame.iat
Acceda a un solo valor para un par de filas / columnas por posición entera.

DataFrame.loc
Acceda a un grupo de filas y columnas por etiqueta (s).

Series.at
Acceda a un único valor mediante una etiqueta. eg: df.at[fila, 'columna']

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.at.html

In [ ]:
for i, cli in enumerate(df_features['Tipo_Cliente_1']):
    ix = df_features['Tipo_Cliente_1'].index[i]
    if df.at[ix, 'DIAS6']  > 30:  # cogemos los calores de las filas ix y columnas Dias6 y selecionamos los que tengan mas de 30 dias
        df_features.at[ix, 'Tipo_Cliente_1'] = 1

df_features['Tipo_Cliente_1'].value_counts()

esto quiere decir que de 781 clientes  731 se precentan como clientes buenos y 50 de estos ya tienes un maximo de 30 dias en mora.

en la siguiente 740 clientes des pues de 6 meses no tienen mas de 90 dias de mora pero 42 clientes tienes 41 dias de mora

In [ ]:
#Clientes que que luego de los 6 meses hayan alcanzado un máximo de 90 días en mora.
df_features['Tipo_Cliente_2'] = 0

for i, cli in enumerate(df_features['Tipo_Cliente_2']):
    ix = df_features['Tipo_Cliente_2'].index[i]
    if df.at[ix, 'DIAS6']  >= 90:
        df_features.at[ix, 'Tipo_Cliente_2'] = 1

df_features['Tipo_Cliente_2'].value_counts()

In [ ]:
#Clientes que que luego de los 6 meses hayan alcanzado un máximo de 180 días en mora.
df_features['Tipo_Cliente_3'] = 0

for i, cli in enumerate(df_features['Tipo_Cliente_3']):
    ix = df_features['Tipo_Cliente_3'].index[i]
    if df.at[ix, 'DIAS6']  >= 180:
        df_features.at[ix, 'Tipo_Cliente_3'] = 1

df_features['Tipo_Cliente_3'].value_counts()

In [ ]:
# esto nos indica los clientes que tienen mora despues de 6 meses, por ejemplo 121.0 clientes tienen 1 de mora
D_mora=df["DIAS6"].value_counts()
print(TOTALES)
# D_mora.plot.barh() # DataFrame.plot.barh( x = Ninguno , y = Ninguno , ** kwargs ) para graficar varras horizontales


# Model Training

In [ ]:
#Variable categorica
y_1 = df_features['Tipo_Cliente_1'].values
y_2 = df_features['Tipo_Cliente_2'].values
y_3 = df_features['Tipo_Cliente_3'].values

In [ ]:
features_num = len(df_features.columns)
features_num

In [ ]:
#Caracterisiticas del problema 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X = df_features.iloc[:,:features_num].values # selecionamos todas las columnas de df_features
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
X.shape

In [ ]:
#Entranamiento del problema
from sklearn.model_selection import train_test_split # para separacion en datos de test y entrenamiento

In [ ]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y_1, test_size=0.33, random_state=42,
                                                            stratify=y_1, shuffle=True)

In [ ]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y_2, test_size=0.33, random_state=42,
                                                            stratify=y_2, shuffle=True)

In [ ]:
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y_3, test_size=0.33, random_state=42,
                                                            stratify=y_3, shuffle=True)

In [ ]:
grid = [i for i in range(1, 11, 1)]
param_grid = {'C': [np.round((0.1*i), decimals =2) for i in grid]}
print('Parameters:', param_grid)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

clf = GridSearchCV(LogisticRegression(solver= 'liblinear'), # solver : {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'},  default='lbfgs' Algorithm to use in the optimization problem.
                   param_grid = param_grid, verbose=0, return_train_score=True, 
                   n_jobs=-1, cv=5)

# Models Evaluation

In [ ]:
class_names = np.array(['Buen Cliente', 'Mal Cliente'])

In [ ]:
clf.fit(X_train_1, y_train_1) 

print('Best Params:', clf.best_params_, '\n')
y_pred_1 = clf.predict(X_test_1)
print(classification_report(y_test_1, y_pred_1))
# The support is the number of occurrences of each class in y_true

In [ ]:
from sklearn.metrics import confusion_matrix
title = "Confusion matrix"
fig, ax = plt.subplots(figsize=(8, 8))
plot_confusion_matrix(clf, X_test_1, y_test_1,
                      display_labels = class_names, # para las etiquetas
                      cmap = plt.cm.Blues, # para elmapa de colores
                      normalize = None, ax=ax)
ax.set_title(title)
plt.plot()
plt.savefig("Confusion matrix modelo1.jpg", bbox_inches='tight') # para guardar como jpg
plt.savefig("Confusion matrix modelo1.png", bbox_inches='tight') # para guardar como png
# plt.savefig('results_Model_1.png', format='png', bbox_inches='tight')

# A continuación se observa la matriz de confusión para el primer modelo (mora de 30 días)

In [ ]:
clf.fit(X_train_2, y_train_2) 

print('Best Params:',clf.best_params_, '\n')
y_pred_2 = clf.predict(X_test_2)
print(classification_report(y_test_2, y_pred_2))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plot_confusion_matrix(clf, X_test_2, y_test_2,
                      display_labels = class_names,
                      cmap = plt.cm.Blues,
                      normalize = None, ax=ax)
ax.set_title(title)
plt.plot()
plt.savefig('Confusion matrix modelo2.png', format='png', bbox_inches='tight')

# para el segundo modelo (mora de 90 días), se obtiene:
# donde se optiene que el modelo predice TP 244 y FN 14

In [ ]:
clf.fit(X_train_3, y_train_3) 

print('Best Params:', clf.best_params_, '\n')
y_pred_3 = clf.predict(X_test_3)
print(classification_report(y_test_3, y_pred_3))
# The support is the number of occurrences of each class in y_true

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plot_confusion_matrix(clf, X_test_3, y_test_3,
                      display_labels = class_names,
                      cmap = plt.cm.Blues,
                      normalize = None, ax=ax)
ax.set_title(title)
plt.plot()
plt.savefig('Confusion matrix modelo2.png', format='png', bbox_inches='tight')

# Y finalmente para el tercer modelo (mora de 180 días), se obtiene:
# donde se optiene que el modelo predice TP 249 y FN 9

In [ ]:
df.head()

In [ ]:
# para ver que tipo de variables tiene las columnas
categori_columna_x=df.DIAS6.unique()
categori_columna_x

In [ ]:
df_features['Porc_prom_Saldo_mora']